In [1]:
import os 
from sqlalchemy import create_engine
import pandas as pd

user1 = os.environ['PAYMEE_SONTATA_DB_USER']
password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
db = "Sonata_Connect"
server = "172.17.130.216"   

engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")


pd.set_option('display.max_columns', 500)

sql_query = f"""SET NOCOUNT ON;SELECT task_id, task_description, status, priority, assigned_to, created_by, completion_date, created_on, deadline_date, category, review, feedback FROM [{db}].[dbo].accounts_task_details ORDER BY created_on;SET NOCOUNT OFF"""
all_tasks_df=pd.read_sql_query(sql_query,engine)


sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 6892;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)

In [2]:
div_df = df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = df[(df['RoleId']==35)&(df['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
hub_df = df[(df['RoleId']==36)&(df['RoleName']=='HUB HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName'}, inplace=True)
bm_df = df[(df['RoleId']==13)&(df['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = df[(df['RoleId']==55)&(df['RoleName']=='BRO')][['U_BUID','UserID','UserName']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

In [3]:
reqdf = pd.merge(all_tasks_df, df, how='left', left_on='assigned_to', right_on='UserID').drop_duplicates(subset=['task_id'])
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )


In [4]:
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,3366.0,rajesh Sahu,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division


In [33]:
reqdf[reqdf['Div_UserID'] == 7662]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
17,129133,Customer (Customer id: Mbd:Chd:03:316) has com...,Completed,1,31693,30892,2023-04-11,2023-04-01,2023-04-01,10,None,Wrong number,45.0,5.0,595.0,Chandpur,31693.0,VIKASH KUMAR,45.0,55.0,BRO,45.0,31693.0,VIKASH KUMAR,45.0,Chandpur,216.0,VIKAS KUMAR,595.0,595.0,16535.0,Kamlesh singh,Joya,1005.0,1005.0,1968.0,sompal,Moradabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
26,129142,Customer (Customer id: Mbd:Chd:03:280) is inte...,Completed,1,31693,30892,2023-04-11,2023-04-01,2023-04-01,7,None,None,45.0,5.0,595.0,Chandpur,31693.0,VIKASH KUMAR,45.0,55.0,BRO,45.0,31693.0,VIKASH KUMAR,45.0,Chandpur,216.0,VIKAS KUMAR,595.0,595.0,16535.0,Kamlesh singh,Joya,1005.0,1005.0,1968.0,sompal,Moradabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
88,129204,Customer (Customer id: GZB:SKD:01:059:044) wil...,Completed,1,17607,30892,2023-04-12,2023-04-01,2023-04-05,3,None,None,281.0,5.0,589.0,Sikandrabad,17607.0,PRASANT KUMAR,281.0,55.0,BRO,281.0,17607.0,PRASANT KUMAR,281.0,Sikandrabad,3064.0,RAHUL SINGH,589.0,589.0,7118.0,Arvind kumar,Ghaziabad,1011.0,1011.0,14050.0,ASHOK KUMAR YADAV,Ghaziabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
113,129229,Customer (Customer id: Bld:Bds:01:147:023) wil...,Completed,1,31409,30892,2023-04-12,2023-04-01,2023-04-01,3,None,None,210.0,5.0,2128.0,Bulandshahr,31409.0,KUMARI KUMKUM,210.0,55.0,BRO,210.0,31409.0,KUMARI KUMKUM,210.0,Bulandshahr,4191.0,VEERI SINGH,2128.0,2128.0,2670.0,AADESH KUMAR,Bulandshahar,1011.0,1011.0,14050.0,ASHOK KUMAR YADAV,Ghaziabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
243,129359,Customer (Customer id: 2815159) will pay an am...,Completed,1,31554,31554,2023-04-11,2023-04-01,2023-04-04,3,None,None,34.0,5.0,593.0,Bareilly,31554.0,PRIYANKA SINGH,34.0,55.0,BRO,34.0,31554.0,PRIYANKA SINGH,34.0,Bareilly,2175.0,RAKESH KUMAR,593.0,593.0,4583.0,SUNEEL KUMAR,Bareilly,2179.0,2179.0,7664.0,Vikash Singh,Bareilly,654.0,654.0,7662.0,Sandeep Singh,Moradabad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28747,172667,Customer (Customer id: 2920461) will pay an am...,Pending,1,28055,28055,None,2023-04-13,2023-04-13,3,None,None,164.0,5.0,594.0,Nageena,28055.0,KUM HIMANSHI,164.0,55.0,BRO,164.0,28055.0,KUM HIMANSHI,164.0,Nageena,3954.0,ANSAR,594.0,594.0,17648.0,SOM VEER SINGH,Bijnor,1005.0,1005.0,1968.0,sompal,Moradabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
28754,172674,Customer (Customer id: 2609718) will pay an am...,Pending,1,28055,28055,None,2023-04-13,2023-04-17,3,None,None,164.0,5.0,594.0,Nageena,28055.0,KUM HIMANSHI,164.0,55.0,BRO,164.0,28055.0,KUM HIMANSHI,164.0,Nageena,3954.0,ANSAR,594.0,594.0,17648.0,SOM VEER SINGH,Bijnor,1005.0,1005.0,1968.0,sompal,Moradabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
28758,172678,Update Phone Number of Customer ID 1012018,Pending,1,31706,31706,None,2023-04-13,2023-04-13,10,None,None,434.0,5.0,2128.0,Khurja,31706.0,KOSHINDER SINGH,434.0,55.0,BRO,434.0,31706.0,KOSHINDER SINGH,434.0,Khurja,4724.0,Dhanesh kumar,2128.0,2128.0,2670.0,AADESH KUMAR,Bulandshahar,1011.0,1011.0,14050.0,ASHOK KUMAR YADAV,Ghaziabad,654.0,654.0,7662.0,Sandeep Singh,Moradabad
28766,172686,Customer (Customer id: 2580113) will pay an am...,Pending,1,30403,30403,None,2023-04-13,2023-04-15,3,None,None,434.0,5.0,2128.0,Khurja,30403.0,Lakshmi Chand,434.0,55.0,BRO,434.0,30403.0,Lakshmi Chand,434.0,Khurja,4724.0,Dhanesh kumar,2128.0,2128.0,2670.0,AADESH KUMAR,Bulandshahar,1011.0,1011.0,14050.0,ASHOK KUMAR YADAV

In [11]:
df.columns

Index(['BUId', 'BUType', 'ReportingBUId', 'buname', 'UserID', 'UserName',
       'U_BUID', 'RoleId', 'RoleName'],
      dtype='object')

In [32]:
type(reqdf['created_on'].iloc[0])

datetime.date

In [41]:
reqdf['priority'].value_counts()

1    28870
4       19
3       18
2       14
Name: priority, dtype: int64

In [42]:
reqdf[reqdf['priority'] == 2]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
20466,164374,test desc 2,Pending,2,31678,7535,None,2023-04-07,2023-04-09,1,None,None,6.0,5.0,553.0,Rudauli,31678.0,ABHISHEK DUBEY,6.0,55.0,BRO,6.0,31678.0,ABHISHEK DUBEY,6.0,Rudauli,1564.0,Virendra kumar Maurya,553.0,553.0,25094.0,ABHISHEK SINGH,FAIZABAD,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
20470,164378,test desc,Pending,2,31678,7535,None,2023-04-07,2023-04-09,1,None,None,6.0,5.0,553.0,Rudauli,31678.0,ABHISHEK DUBEY,6.0,55.0,BRO,6.0,31678.0,ABHISHEK DUBEY,6.0,Rudauli,1564.0,Virendra kumar Maurya,553.0,553.0,25094.0,ABHISHEK SINGH,FAIZABAD,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
20877,164789,test,Pending,2,30914,7535,None,2023-04-07,2023-04-20,14,None,None,17.0,5.0,586.0,Mishrikh,30914.0,AMRENDRA KUMAR GUPTA,17.0,55.0,BRO,17.0,30914.0,AMRENDRA KUMAR GUPTA,17.0,Mishrikh,15057.0,Surendra Kumar,586.0,586.0,7197.0,abhishek kumar singh,sitapur,2292.0,2292.0,0.0,REGION HEAD NOT POSTED,Sitapur,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
22896,166812,test,Closed,2,30307,30307,2023-04-10,2023-04-10,2023-04-10,3,None,None,419.0,5.0,554.0,Dhaneypur,30307.0,Shiv Kumar,419.0,55.0,BRO,419.0,30307.0,Shiv Kumar,419.0,Dhaneypur,3988.0,Santosh Kumar,554.0,554.0,2399.0,Rakesh Kumar Pal,GONDA,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
23267,167183,test,Pending,2,7535,7535,None,2023-04-10,2023-04-10,3,None,None,651.0,2.0,1.0,Lucknow Division,7535.0,RAKESH KUMAR CHAUDHARY,651.0,34.0,DIVISION HEAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23322,167238,form,Pending,2,9739,9739,None,2023-04-10,2023-04-10,3,None,None,650.0,2.0,1.0,Prayagraj Division,9739.0,LAKSHMAN SINGH,650.0,34.0,DIVISION HEAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24680,168596,test,Closed,2,27784,27784,2023-04-11,2023-04-11,2023-04-12,15,2,1,419.0,5.0,554.0,Dhaneypur,27784.0,Suraj Kumar Yadav,419.0,55.0,BRO,419.0,27784.0,Suraj Kumar Yadav,419.0,Dhaneypur,3988.0,Santosh Kumar,554.0,554.0,2399.0,Rakesh Kumar Pal,GONDA,1006.0,1006.0,18033.0,CHANDRA SHEKHAR,Faizabad,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
24779,168695,You have to describe task here whatever be the...,Re-Schedule,2,30823,30823,None,2023-04-11,2023-04-20,3,None,None,2244.0,5.0,2132.0,Aurai,30823.0,SAVITA,2244.0,55.0,BRO,2244.0,30823.0,SAVITA,2244.0,Aurai,1597.0,RAJU KUMAR MAURYA,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
25003,168919,flejgejgsjg sgsgjgorjgorhun ahgiehghowihg,Pending,2,30823,30823,None,2023-04-11,2023-04-12,10,None,None,2244.0,5.0,2132.0,Aurai,30823.0,SAVITA,2244.0,55.0,BRO,2244.0,30823.0,SAVITA,2244.0,Aurai,1597.0,RAJU KUMAR MAURYA,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
25189,169105,scs,Pending,2,30823,30823,None,2023-04-11,2023-04-26,13,None,None,2244.0,5.0,2132.0,Aurai,30823.0,SAVITA,2244.0,55.0,BRO,2244.0,30823.0,SAVITA,2244.0,Aurai,1597.0,RAJU KUMAR MAURYA,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division


In [43]:
reqdf.columns

Index(['task_id', 'task_description', 'status', 'priority', 'assigned_to',
       'created_by', 'completion_date', 'created_on', 'deadline_date',
       'category', 'review', 'feedback', 'BUId', 'BUType', 'ReportingBUId',
       'buname', 'UserID', 'UserName', 'U_BUID', 'RoleId', 'RoleName',
       'BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [46]:
type(reqdf['deadline_date'].iloc[0])

datetime.date

In [47]:
from datetime import datetime

In [50]:
type(datetime.now().date())

datetime.date

In [57]:
reqdf[reqdf['priority'] == int('1')]

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Pending,1,29938,2271,None,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,3366.0,rajesh Sahu,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28916,172836,Customer (Customer id: Muz:Muz:2059:037:023) w...,Pending,1,28282,27529,None,2023-04-13,2023-04-18,3,None,None,2059.0,5.0,2171.0,Lalganj_MZF,28282.0,Raushan Kumar,2059.0,55.0,BRO,2059.0,28282.0,Raushan Kumar,2059.0,Lalganj_MZF,4164.0,ARUN KUMAR,2171.0,2171.0,7211.0,Krishna Kumar,KURHANI,1012.0,1012.0,24776.0,RAJNAND MAURYA,Muzaffarpur,653.0,653.0,10849.0,KESHRAJ YADAV,Muzaffarpur
28917,172837,Customer (Customer id: NA) is interested in a ...,Pending,1,10854,27185,None,2023-04-13,2023-04-13,7,None,None,68.0,5.0,2132.0,DurgaGanj,10854.0,Ajeet kumar yadav,68.0,55.0,BRO,68.0,10854.0,Ajeet kumar yadav,68.0,DurgaGanj,1417.0,YOGENDRA KUMAR,2132.0,2132.0,3883.0,RAVINDRA KUMAR,HANDIA,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
28918,172838,Customer (Customer id: Ald:Php:02:663) is inte...,Pending,1,27528,27185,None,2023-04-13,2023-04-13,7,None,None,57.0,5.0,549.0,Hanumanganj,27528.0,Shartrughan Kumar,57.0,55.0,BRO,57.0,27528.0,Shartrughan Kumar,57.0,Hanumanganj,24903.0,UMESH KUMAR MAURYA,549.0,549.0,1466.0,Vikash Kumar Maurya,PHOOLPUR,1001.0,1001.0,7114.0,Akash Jyoti singh,Prayagraj,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
28919,172839,Customer (Customer id: BUX:BUX:2185:071:002) w...,Pending,1,31530,27518,None,2023-04-13,2023-04-19,3,None,None,2185.0,5.0,2062.0,Nuaon,31530.0,MUKESH

In [59]:
cols_req = [
        'created_on',
        'DivName',
        'RegionName',
        'HubName',
        'BranchName',
        'assigned_to',
        'created_by',
        'priority',
        'task_id',
        'category',
        'task_description',
        'deadline_date',
        #reschedule date
        #whatsapp conf
        'status',
        'completion_date',
        'review',
        'feedback'
    ]

In [60]:
reqdf = reqdf.sort_values(by=['created_on', 'DivName', 'RegionName', 'HubName', 'BranchName'], ascending=False)[cols_req]

In [61]:
reqdf

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
28445,2023-04-13,Varanasi,Varanasi,Varanasi,Varanasi city,16354,11905,1,172365,3,Customer (Customer id: Var:Var:2002:2903262) w...,2023-04-17,Pending,None,None,None
28510,2023-04-13,Varanasi,Varanasi,Varanasi,Varanasi city,30109,30109,1,172430,3,Customer (Customer id: 2427649) will pay an am...,2023-04-16,Pending,None,None,None
28527,2023-04-13,Varanasi,Varanasi,Varanasi,Varanasi city,30109,30109,1,172447,3,Customer (Customer id: 2548353) will pay an am...,2023-04-17,Pending,None,None,None
28820,2023-04-13,Varanasi,Varanasi,Varanasi,Varanasi city,30109,30109,1,172740,3,Customer (Customer id: 2803763) will pay an am...,2023-04-14,Pending,None,None,None
28813,2023-04-13,Varanasi,Varanasi,Varanasi,Pandeypur,30672,11905,1,172733,3,Customer (Customer id: Var:Pan:444:240:006) wi...,2023-04-13,Pending,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,2023-04-01,NaN,NaN,NaN,NaN,31864,17861,1,129819,7,Customer (Customer id: Bha:Bha:239:343) is int...,2023-04-01,Pending,None,None,None
751,2023-04-01,NaN,NaN,NaN,NaN,27508,1473,1,129867,3,Customer (Customer id: Mot:Mot:477:085:014) wi...,2023-04-04,Pending,None,None,None
785,2023-04-01,NaN,NaN,NaN,NaN,26917,1473,1,129901,3,Customer (Customer id: Mot:Mot:477:131:009) wi...,2023-04-06,Pending,None,None,None
799,2023-04-01,NaN,NaN,NaN,NaN,26917,1473,1,129915,3,Customer (Customer id: Ech:Cka:01:039:028) wil...,2023-04-07,Pending,None,None,None


In [91]:
from_date = '2023-04-07'
to_date = '2023-04-13'

In [92]:
datetime.now().date()

datetime.date(2023, 4, 14)

In [93]:
datetime.strptime(from_date, "%Y-%M-%d").date() > datetime.now().date()

False

In [94]:
reqdf[(reqdf['created_on'] >= datetime.strptime(from_date, "%Y-%M-%d").date()) & (reqdf['created_on'] <= datetime.strptime(to_date, "%Y-%M-%d").date())]

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback


In [95]:
re1 = reqdf[(reqdf['created_on'] >= datetime.strptime(from_date, "%Y-%M-%d").date())]
re2 = re1[re1[(re1['created_on'] <= datetime.strptime(to_date, "%Y-%M-%d").date())]]

In [96]:
datetime.strptime(from_date, "%Y-%M-%d").date()

datetime.date(2023, 1, 7)

In [100]:
dF.columns

Index(['BRO_BUID', 'BRO_UserID', 'BRO_UserName', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [97]:
reqdf[(reqdf['deadline_date'] >= datetime.strptime(from_date, "%Y-%m-%d").date()) & (reqdf['deadline_date'] <= datetime.strptime(to_date, "%Y-%m-%d").date())]


,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
28813,2023-04-13,Varanasi,Varanasi,Varanasi,Pandeypur,30672,11905,1,172733,3,Customer (Customer id: Var:Pan:444:240:006) wi...,2023-04-13,Pending,None,None,None
28493,2023-04-13,Varanasi,Varanasi,Varanasi,Chaubeypur,25562,11905,1,172413,10,Customer (Customer id: Var:Var:445:212:015) ha...,2023-04-13,Pending,None,None,None
28836,2023-04-13,Varanasi,Varanasi,Varanasi,Chaubeypur,27484,11905,1,172756,3,Customer (Customer id: Var:Var:445:099:033) wi...,2023-04-13,Pending,None,None,None
28865,2023-04-13,Varanasi,Varanasi,ROBERTSGANJ,ROBERTSGANJ,30367,11905,1,172785,10,Customer (Customer id: Var:Var:2101:008:023) h...,2023-04-13,Pending,None,None,None
28178,2023-04-13,Varanasi,Varanasi,MUGHAL SARAI,Rajatalab,28162,11905,1,172098,7,Customer (Customer id: Var:Var:377:209) is int...,2023-04-13,Pending,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2023-04-01,Gwalior,Gwalior,GWALIOR,Datia City,27186,9738,1,129197,3,Customer (Customer id: Dat:Dat:01:049:022) wil...,2023-04-12,Completed,2023-04-10,None,None
64,2023-04-01,NaN,NaN,NaN,NaN,27508,1473,1,129180,3,Customer (Customer id: Mot:Mot:477:129:027) wi...,2023-04-07,Pending,None,None,None
173,2023-04-01,NaN,NaN,NaN,NaN,27508,1473,1,129289,3,Customer (Customer id: Mot:Mot:477:113:017) wi...,2023-04-07,Pending,None,None,None
682,2023-04-01,NaN,NaN,NaN,NaN,26917,1473,1,129798,3,Customer (Customer id: Mot:Mot:477:185:010) wi...,2023-04-07,Pending,None,None,None


In [98]:
reqdf[reqdf['priority'] == 3]

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
28589,2023-04-13,Jabalpur,CHINDWADA,CHINDWADA,Umarnala,27625,27625,3,172509,12,Customer Sangita Tayvade Want a New Loan Of Am...,2023-10-16,Pending,None,None,None
26304,2023-04-12,Saharanpur,Chandigarh,Rajpura,Sangrur,27337,12021,3,170220,3,intallment,2023-04-21,Re-Schedule,None,None,None
24834,2023-04-11,Prayagraj Division,Prayagraj,HANDIA,Aurai,30823,30823,3,168750,6,cvmnsbhksfjb fsjbhsljhbsxbmxkcfb,2023-04-23,Pending,None,None,None
25168,2023-04-11,Prayagraj Division,Prayagraj,HANDIA,Aurai,30823,30823,3,169084,13,kdkhkhvdh dvkdhkdhbk vkdhkvhbkdhbk vkhbkhsbkh ...,2023-04-19,Completed,2023-04-11,Nonesvajb JBCKBC Kbcksbccdkd,chcsjcosjv knvkanvka kvha
25245,2023-04-11,Prayagraj Division,Prayagraj,HANDIA,Aurai,30823,30823,3,169161,15,efbjfb jbjabvak kvnakdgv,2023-04-25,Pending,None,None,None
25272,2023-04-11,Prayagraj Division,Prayagraj,HANDIA,Aurai,30823,30823,3,169188,16,shsrhsh,2023-05-05,Pending,None,None,None
22888,2023-04-10,Lucknow Division,Faizabad,GONDA,Dhaneypur,27784,27784,3,166804,4,sample task,2023-04-10,Closed,2023-04-10,None,None
23306,2023-04-10,NaN,NaN,NaN,NaN,7665,7665,3,167222,7,self,2023-04-10,Pending,None,None,None
20558,2023-04-07,Lucknow Division,Kanpur,Billaur,Billhor,18059,7535,3,164466,4,rm,2023-04-30,Closed,None,None,None
21056,2023-04-07,Gwalior,Sagar,Hata,Panna City,27681,3056,3,164968,12,Customer Priti kushwaha Want a New Loan Of A...,2023-04-07,Pending,None,None,None


In [106]:
d = dict(zip(df['UserID'], df['UserName']))
reqdf['assigned_to'] = reqdf['assigned_to'].apply(lambda x : str(x) + '-' + d[x] if x in d.keys() else str(x))

In [114]:
reqdf[(reqdf['BranchName'] == 'Katangi')]

,created_on,DivName,RegionName,HubName,BranchName,assigned_to,created_by,priority,task_id,category,task_description,deadline_date,status,completion_date,review,feedback
28409,2023-04-13,Jabalpur,CHINDWADA,SEONI,Katangi,31545-VISHAL BISEN,29968,1,172329,3,Customer (Customer id: CHI:SEO:2127:024:005) w...,2023-04-18,Pending,None,None,None
28902,2023-04-13,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,29968,1,172822,3,Customer (Customer id: CHI:SEO:2127:049:004) w...,2023-04-18,Pending,None,None,None
26613,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31545-VISHAL BISEN,29968,1,170529,10,Customer (Customer id: CHI:SEO:2127:011:019) h...,2023-04-12,Pending,None,None,None
27404,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171320,3,Customer (Customer id: 3210597) will pay an am...,2023-04-14,Pending,None,None,None
27405,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171321,3,Customer (Customer id: 3210597) will pay an am...,2023-04-14,Pending,None,None,None
27406,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171322,3,Customer (Customer id: 3210597) will pay an am...,2023-04-14,Pending,None,None,None
27407,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171323,3,Customer (Customer id: 3210597) will pay an am...,2023-04-14,Pending,None,None,None
27408,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171324,3,Customer (Customer id: 3210597) will pay an am...,2023-04-14,Pending,None,None,None
27410,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171326,3,Customer (Customer id: 2867709) will pay an am...,2023-04-14,Pending,None,None,None
27413,2023-04-12,Jabalpur,CHINDWADA,SEONI,Katangi,31371-DEVID HARINKHEDE,31371,1,171329,3,Customer (Customer id: 2867709) will pay an am...,2023-04-14,Pending,None,None,None


In [115]:
new_columns = ['Assigned_Date','Division','Region','Hub', 'Branch', 'Assigned_To_User', 'Assigned_By', 'Priority', 'TaskID', 'Category', 'Task_Description', 'Target_Date', 'Status', 'Close_Date', 'Review', 'Feedback']
len(new_columns)

16

In [116]:
len(reqdf.columns)

16